In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import numpy as np
import os
import selfies as sf
import exmol
import itertools
soldata = pd.read_csv('https://github.com/whitead/dmol-book/raw/master/data/curated-solubility-dataset.csv')
features_start_at = list(soldata.columns).index('MolWt')
np.random.seed(0)

In [2]:
# scramble them
# Reduced for CI!
soldata = soldata.sample(frac=0.15, random_state=0).reset_index(drop=True)
soldata.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,B-4206,diuron,InChI=1S/C9H10Cl2N2O/c1-13(2)9(14)12-6-3-4-7(1...,XMTQQYYKAHVGBJ-UHFFFAOYSA-N,CN(C)C(=O)Nc1ccc(Cl)c(Cl)c1,-3.744300,1.227164,5,G4,233.098,...,1.0,76.0,1.0,0.0,0.0,1.0,32.34,92.603980,2.781208,352.665233
1,F-988,7-(3-amino-3-methylazetidin-1-yl)-8-chloro-1-c...,InChI=1S/C17H17ClFN3O3/c1-17(20)6-21(7-17)14-1...,DUNZFXZSFJLIKR-UHFFFAOYSA-N,CC1(N)CN(C2=C(Cl)C3=C(C=C2F)C(=O)C(C(=O)O)=CN3...,-5.330000,0.000000,1,G1,365.792,...,3.0,132.0,2.0,2.0,2.0,4.0,88.56,147.136366,2.001398,973.487509
2,C-1996,4-acetoxybiphenyl; 4-biphenylyl acetate,InChI=1S/C14H12O2/c1-11(15)16-14-9-7-13(8-10-1...,MISFQCBPASYYGV-UHFFFAOYSA-N,CC(=O)OC1=CC=C(C=C1)C2=CC=CC=C2,-4.400000,0.000000,1,G1,212.248,...,2.0,80.0,2.0,0.0,0.0,2.0,26.30,94.493449,2.228677,471.848345
3,A-3055,methane dimolybdenum,InChI=1S/CH4.2Mo/h1H4;;,JAGQSESDQXCFCH-UHFFFAOYSA-N,C.[Mo].[Mo],-3.420275,0.409223,2,G3,207.923,...,0.0,20.0,0.0,0.0,0.0,0.0,0.00,49.515427,-0.000000,2.754888
4,A-2575,ethyl 4-[[(methylphenylamino)methylene]amino]b...,InChI=1S/C17H18N2O2/c1-3-21-17(20)14-9-11-15(1...,GNGYPJUKIKDJQT-UHFFFAOYSA-N,CCOC(=O)c1ccc(cc1)N=CN(C)c2ccccc2,-5.450777,0.000000,1,G1,282.343,...,5.0,108.0,2.0,0.0,0.0,2.0,41.90,124.243431,2.028889,606.447052


In [3]:
selfies_list = [sf.encoder(exmol.sanitize_smiles(s)[1]) for s in soldata.SMILES]

In [4]:
len(selfies_list)

1497

In [5]:
#selfies_list = [''.join(list(sf.split_selfies(s))[:5]) for s in selfies_list]

In [6]:
symbol_counts = {}
        
for s in selfies_list:
    #print(s)
    s_list = list(sf.split_selfies(s))    
    for token in s_list:
        if '.' in token:
            continue #?
        if token in symbol_counts:
            symbol_counts[token] += 1
        else:
            symbol_counts[token] = 1

symbol_counts = dict(sorted(symbol_counts.items(), key=lambda i: i[1]))
print(symbol_counts)
print()

vocab = list(symbol_counts.keys())
print(vocab)
print()
vocab_stoi = {o:i for o,i in zip(vocab, range(len(vocab)))}
print(vocab_stoi)
print()


{'[Zrexpl]': 1, '[Cd+2expl]': 1, '[Seexpl]': 1, '[Sr+2expl]': 1, '[=Seexpl]': 1, '[Reexpl]': 1, '[Znexpl]': 1, '[=O+expl]': 1, '[Vexpl]': 1, '[SiHexpl]': 1, '[Mn+2expl]': 1, '[P+expl]': 1, '[I-expl]': 1, '[Mn+3expl]': 1, '[=Crexpl]': 1, '[=Feexpl]': 1, '[Mnexpl]': 1, '[=Znexpl]': 1, '[Laexpl]': 1, '[N-expl]': 1, '[Cu+2expl]': 1, '[=Cdexpl]': 1, '[Zr+2expl]': 1, '[Cr+3expl]': 1, '[=Coexpl]': 1, '[Hgexpl]': 1, '[Hg+expl]': 1, '[Y+3expl]': 1, '[=I]': 1, '[=Moexpl]': 1, '[Kexpl]': 1, '[Liexpl]': 1, '[Tiexpl]': 1, '[=Ceexpl]': 1, '[B-expl]': 1, '[Bexpl]': 1, '[=Caexpl]': 1, '[NH3+expl]': 1, '[Alexpl]': 1, '[Asexpl]': 1, '[NH2+expl]': 1, '[=Siexpl]': 1, '[MgH2expl]': 1, '[Hf+4expl]': 1, '[Rh+3expl]': 1, '[PHexpl]': 1, '[Ni+2expl]': 1, '[Auexpl]': 1, '[Cu+expl]': 1, '[Fe+2expl]': 1, '[C+expl]': 2, '[Zn+2expl]': 2, '[Lu+3expl]': 2, '[Mg+2expl]': 2, '[Cuexpl]': 2, '[=Pbexpl]': 2, '[Bi+3expl]': 2, '[=Mnexpl]': 2, '[H-expl]': 2, '[Clexpl]': 2, '[Sbexpl]': 2, '[Al+3expl]': 2, '[Expl=Ring2]': 2, '[

In [7]:
#Using vocab_stoi ordered by frequency for convenience

#vocab = list(sf.get_alphabet_from_selfies([s for s in selfies_list if s is not None]))
#vocab_stoi = {o:i for o,i in zip(vocab, range(len(vocab)))}
#print('Vocab size', len(vocab))

In [8]:
def selfies2ints(s):
    result = []
    for token in sf.split_selfies(s):
        if token == '.':
            continue #?
        if token in vocab_stoi:
            result.append(vocab_stoi[token])
        else:
            result.append(np.nan)
            #print('Warning')
    return result
def ints2selfies(v):
    return ''.join([vocab[i] for i in v])

# test them out
s = selfies_list[0]
print('selfies:', s)
v = selfies2ints(s)
print('selfies2ints:', v)
so = ints2selfies(v)
print('ints2selfes:', so)
assert so == s

selfies: [C][N][Branch1_1][C][C][C][Branch1_2][C][=O][N][C][=C][C][=C][Branch1_1][C][Cl][C][Branch1_1][C][Cl][=C][Ring1][Branch2_1]
selfies2ints: [118, 111, 116, 118, 118, 118, 114, 118, 112, 111, 118, 117, 118, 117, 116, 118, 109, 118, 116, 118, 109, 117, 115, 108]
ints2selfes: [C][N][Branch1_1][C][C][C][Branch1_2][C][=O][N][C][=C][C][=C][Branch1_1][C][Cl][C][Branch1_1][C][Cl][=C][Ring1][Branch2_1]


In [9]:
ints_list = [selfies2ints(s) for s in selfies_list]

In [10]:
max_len = max([len(i) for i in ints_list])
max_len

200

## Shapley for 1-gram model

In [11]:
uni_prob_vector = np.ones(len(vocab))/len(vocab)
print(uni_prob_vector)
print(len(uni_prob_vector))

[0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336
 0.00840336 0.00840336 0.00840336 0.00840336 0.00840336 0.0084

In [12]:
freq_prob_vector = list(symbol_counts.values())
freq_prob_vector = freq_prob_vector/np.sum(freq_prob_vector)
print(freq_prob_vector)
print(len(freq_prob_vector))

[2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 2.49681656e-05 2.49681656e-05
 2.49681656e-05 2.49681656e-05 4.99363312e-05 4.99363312e-05
 4.99363312e-05 4.99363312e-05 4.99363312e-05 4.99363312e-05
 4.99363312e-05 4.99363312e-05 4.99363312e-05 4.99363312e-05
 4.99363312e-05 4.99363312e-05 4.99363312e-05 4.99363312e-05
 4.99363312e-05 7.490449

In [13]:
instance = selfies_list[5]
instance_ints = selfies2ints(instance)
instance_len = len(instance_ints)
print(instance)
print(instance_len)
print(instance_ints)
print()

[N][#C][C][C][O]
5
[111, 99, 118, 118, 113]



In [14]:
np.random.seed(100)
print(np.argmax(np.random.multinomial(1, uni_prob_vector, size=instance_len),axis=-1))
np.random.seed(100)
print(np.argmax(np.random.multinomial(1, freq_prob_vector, size=instance_len),axis=-1))

[83 53  0  0  6]
[113 115 116 102 118]


In [15]:
def get_samples(instance_ints, steps, prob_vector, seed = np.random.randint(1000)):
    np.random.seed(seed)
    instance_len = len(instance_ints)
    samples = np.stack([np.argmax(np.random.multinomial(1, prob_vector, size=instance_len),axis=-1) for i in range(steps)])
    return samples

print(get_samples(instance_ints,5,uni_prob_vector, seed=0))
print(get_samples(instance_ints,5,freq_prob_vector, seed=0))
    

[[ 52  96  60  75 108]
 [ 25  29  26  83  32]
 [ 23 107  38  98  85]
 [ 22  53  75  20  94]
 [  1  62  12  20  77]]
[[116 118 114 118 118]
 [118 118 117 117 116]
 [110 111 118 110 117]
 [113  98 118 114 117]
 [112 113 114  98  86]]


In [16]:
uni_samples = get_samples(instance_ints, 2000, uni_prob_vector, seed=0)
freq_samples = get_samples(instance_ints, 2000, freq_prob_vector, seed=0)

In [17]:
def model_1gram(ints): # assuming input is an array of ints
    out = np.sum(ints,axis=1) # sum of each row in ints
    return out

In [18]:
uni_pred = model_1gram(uni_samples)
uni_expec_val = np.sum(uni_pred)/len(uni_pred)
print(uni_expec_val)

295.0455


In [19]:
freq_pred = model_1gram(freq_samples)
freq_expec_val = np.sum(freq_pred)/len(freq_pred)
print(freq_expec_val)

572.596


In [20]:
def shapley(instance_ints, feat, samples, model, debug=False):
    instance_len = len(instance_ints)
    act_len = len([i for i in instance_ints if i!=0])
    #print(act_len)
    if feat >= instance_len:
        print("Error: feat >= seq_len")
        return exit(1)
    feat_int = instance_ints[feat]
    if debug: print("feat_int: ",feat_int)

    #uses all 
    seq_ref = np.stack([s for s in samples])
    
    if debug: 
        print("seq_ref")
        print(seq_ref)
        print()
    
    seq_p = seq_ref.copy()
    seq_m = seq_ref.copy()    
    # set new_x[:] i to be x_i
    seq_p[:,feat] = feat_int
    if debug: 
        print("seq_p")
        print(seq_p)
        print()
        print("seq_m")
        print(seq_m)
        print()
    
    val_p = model(seq_p)
    val_m = model(seq_m)
    if debug: 
        print("val_p")
        print(val_p)
        print()
        print("val_m")
        print(val_m)
        print()
        print("diff")
        print(val_p - val_m)
        print()
    
    result = np.sum(val_p-val_m)/len(samples)
    
    return result

In [21]:
instance = selfies_list[5]
instance_ints = selfies2ints(instance)
instance_len = len(instance_ints)

feat = 1
steps = 2000 # when using selifes_list/ints_list, make sure steps <= len(selfies_list)
model = model_1gram

print("instance: ",instance)
print("instance ints: ",instance_ints)
print("instance len: ",instance_len)
print()

uni_samples = get_samples(instance_ints, steps, uni_prob_vector, seed=0)
freq_samples = get_samples(instance_ints, steps, freq_prob_vector, seed=0)

print(shapley(instance_ints, feat, uni_samples, model))
print()
print(shapley(instance_ints, feat, freq_samples, model))

instance:  [N][#C][C][C][O]
instance ints:  [111, 99, 118, 118, 113]
instance len:  5

40.039

-15.586


In [22]:
instance = selfies_list[5]
instance_ints = selfies2ints(instance)
instance_len = len(instance_ints)

feat = 1
steps = 2000 # when using selifes_list/ints_list, make sure steps <= len(selfies_list)
model = model_1gram

print("instance: ",instance)
print("instance ints: ",instance_ints)
print("instance len: ",instance_len)
print()

uni_samples = get_samples(instance_ints, steps, uni_prob_vector, seed=0)
freq_samples = get_samples(instance_ints, steps, freq_prob_vector, seed=0)

uni_pred = model_1gram(uni_samples)
uni_expec_val = np.sum(uni_pred)/len(uni_pred)

freq_pred = model_1gram(freq_samples)
freq_expec_val = np.sum(freq_pred)/len(freq_pred)

uni_shap_val = [shapley(instance_ints, feat, uni_samples, model) for feat in range(instance_len)]
freq_shap_val = [shapley(instance_ints, feat, freq_samples, model) for feat in range(instance_len)]
print("shapley values uni: ", uni_shap_val)
print("uni Expec: ", uni_expec_val)
print("uni Sum: ", np.sum(uni_shap_val) + uni_expec_val)
print()
print("shapley values freq: ", freq_shap_val)
print("freq Expec: ", freq_expec_val)
print("freq Sum: ", np.sum(freq_shap_val) + freq_expec_val)
print()
label =  model_1gram([instance_ints])[0]
print("f(s): ", label)


instance:  [N][#C][C][C][O]
instance ints:  [111, 99, 118, 118, 113]
instance len:  5

shapley values uni:  [52.6565, 40.039, 59.3235, 58.8475, 53.088]
uni Expec:  295.0455
uni Sum:  559.0

shapley values freq:  [-3.4395, -15.586, 3.6455, 3.478, -1.694]
freq Expec:  572.596
freq Sum:  559.0

f(s):  559


In [23]:
# implementation from https://link.springer.com/content/pdf/10.1007/s10115-013-0679-x.pdf
def shapley_sk(instance_ints, feat, samples, model, debug=False):
    instance_len = len(instance_ints)
    if feat >= instance_len:
        print("Error: feat >= seq_len")
        return exit(1)
    feat_int = instance_ints[feat]
    if debug: print("feat_int: ",feat_int)
        
    seq_ref = np.stack([s for s in samples])
    if debug: 
        print("seq_ref")
        print(seq_ref)
        print()

    # choice
    choice_p = np.stack([np.random.randint(2, size=instance_len) for i in range(len(samples))])
    choice_p[:,feat] = 1
    if debug: 
        print("choice_p")
        print(choice_p)
        print()
    
    choice_m = np.copy(choice_p)
    choice_m[:,feat] = 0
    if debug: 
        print("choice_m")
        print(choice_m)
        print()
    
    # shuffled x's based on choice
    
    seq_p = (instance_ints * choice_p) + (seq_ref * (1-choice_p))
    seq_m = (instance_ints * choice_m) + (seq_ref * (1-choice_m))
    if debug: 
        print("seq_p")
        print(seq_p)
        print()
        print("seq_m")
        print(seq_m)
        print()
    val_p = model(seq_p)
    val_m = model(seq_m)
    if debug: 
        print("val_p")
        print(val_p)
        print()
        print("val_m")
        print(val_m)
        print()
        print("diff")
        print(val_p - val_m)
        print()
    result = np.sum(val_p-val_m)/steps
    
    return result

In [24]:
instance = selfies_list[5]
instance_ints = selfies2ints(instance)
instance_len = len(instance_ints)

feat = 1
steps = 2000 # when using selifes_list/ints_list, make sure steps <= len(selfies_list)
model = model_1gram

print("instance: ",instance)
print("instance ints: ",instance_ints)
print("instance len: ",instance_len)
print()

uni_samples = get_samples(instance_ints, steps, uni_prob_vector, seed=0)
freq_samples = get_samples(instance_ints, steps, freq_prob_vector, seed=0)

print(shapley_sk(instance_ints, feat, uni_samples, model))
print()
print(shapley_sk(instance_ints, feat, freq_samples, model))

instance:  [N][#C][C][C][O]
instance ints:  [111, 99, 118, 118, 113]
instance len:  5

40.039

-15.586


In [25]:
instance = selfies_list[5]
instance_ints = selfies2ints(instance)
instance_len = len(instance_ints)

feat = 1
steps = 5 # when using selifes_list/ints_list, make sure steps <= len(selfies_list)
model = model_1gram

print("instance: ",instance)
print("instance ints: ",instance_ints)
print("instance len: ",instance_len)
print()

uni_samples = get_samples(instance_ints, steps, uni_prob_vector, seed=0)
freq_samples = get_samples(instance_ints, steps, freq_prob_vector, seed=0)

print(shapley(instance_ints, feat, uni_samples, model,True))
print()
print(shapley_sk(instance_ints, feat, uni_samples, model,True))

instance:  [N][#C][C][C][O]
instance ints:  [111, 99, 118, 118, 113]
instance len:  5

feat_int:  99
seq_ref
[[ 52  96  60  75 108]
 [ 25  29  26  83  32]
 [ 23 107  38  98  85]
 [ 22  53  75  20  94]
 [  1  62  12  20  77]]

seq_p
[[ 52  99  60  75 108]
 [ 25  99  26  83  32]
 [ 23  99  38  98  85]
 [ 22  99  75  20  94]
 [  1  99  12  20  77]]

seq_m
[[ 52  96  60  75 108]
 [ 25  29  26  83  32]
 [ 23 107  38  98  85]
 [ 22  53  75  20  94]
 [  1  62  12  20  77]]

val_p
[394 265 343 310 209]

val_m
[391 195 351 264 172]

diff
[ 3 70 -8 46 37]

29.6

feat_int:  99
seq_ref
[[ 52  96  60  75 108]
 [ 25  29  26  83  32]
 [ 23 107  38  98  85]
 [ 22  53  75  20  94]
 [  1  62  12  20  77]]

choice_p
[[1 1 0 0 0]
 [0 1 1 0 0]
 [1 1 1 1 1]
 [1 1 1 1 0]
 [0 1 0 0 1]]

choice_m
[[1 0 0 0 0]
 [0 0 1 0 0]
 [1 0 1 1 1]
 [1 0 1 1 0]
 [0 0 0 0 1]]

seq_p
[[111  99  60  75 108]
 [ 25  99 118  83  32]
 [111  99 118 118 113]
 [111  99 118 118  94]
 [  1  99  12  20 113]]

seq_m
[[111  96  60  75 108

In [59]:
instance = selfies_list[5]
instance_ints = selfies2ints(instance)
instance_len = len(instance_ints)

feat = 1
steps = 2000 # when using selifes_list/ints_list, make sure steps <= len(selfies_list)
model = model_1gram

print("instance: ",instance)
print("instance ints: ",instance_ints)
print("instance len: ",instance_len)
print()

uni_samples = get_samples(instance_ints, steps, uni_prob_vector, seed=0)
freq_samples = get_samples(instance_ints, steps, freq_prob_vector, seed=0)

uni_pred = model_1gram(uni_samples)
uni_expec_val = np.sum(uni_pred)/len(uni_pred)

freq_pred = model_1gram(freq_samples)
freq_expec_val = np.sum(freq_pred)/len(freq_pred)

uni_shap_val = [shapley_sk(instance_ints, feat, uni_samples, model) for feat in range(instance_len)]
freq_shap_val = [shapley_sk(instance_ints, feat, freq_samples, model) for feat in range(instance_len)]
print("shapley values uni: ", uni_shap_val)
print("uni Expec: ", uni_expec_val)
print("uni Sum: ", np.sum(uni_shap_val) + uni_expec_val)
print()
print("shapley values freq: ", freq_shap_val)
print("freq Expec: ", freq_expec_val)
print("freq Sum: ", np.sum(freq_shap_val) + freq_expec_val)
print()
label =  model_1gram([instance_ints])[0]
print("f(s): ", label)


instance:  [N][#C][C][C][O]
instance ints:  [111, 99, 118, 118, 113]
instance len:  5

shapley values uni:  [52.6565, 40.039, 59.3235, 58.8475, 53.088]
uni Expec:  295.0455
uni Sum:  559.0

shapley values freq:  [-3.4395, -15.586, 3.6455, 3.478, -1.694]
freq Expec:  572.596
freq Sum:  559.0

f(s):  559


In [ ]:
shapley values uni:  [52.6565, 40.039, 59.3235, 58.8475, 53.088]
uni Expec:  295.0455
uni Sum:  559.0

shapley values freq:  [-3.4395, -15.586, 3.6455, 3.478, -1.694]
freq Expec:  572.596
freq Sum:  559.0

f(s):  559



In [ ]:
shapley values uni:  [52.6565, 40.039, 59.3235, 58.8475, 53.088]
uni Expec:  295.0455
uni Sum:  559.0

shapley values freq:  [-3.4395, -15.586, 3.6455, 3.478, -1.694]
freq Expec:  572.596
freq Sum:  559.0

f(s):  559